# VOD 마트 데이터 전처리

<br><hr>

### 00. 기본 설정

In [28]:
import pandas as pd
import numpy as np

# 경고 메시지 출력 X
import warnings
warnings.filterwarnings("ignore")

<br><hr>

### 01. 데이터 불러오기

In [3]:
mart = pd.read_csv('../data/mart_0121.csv', encoding='utf8')
print(mart.shape)   # 5만 3874개

(14292, 19)


In [4]:
mart.columns

Index(['asset_id', 'movie_id', 'title', 'original_title', 'genre', 'adult',
       'runtime', 'release_year', 'release_date', 'actors', 'director',
       'orgnl_cntry', 'original_language', 'vote_average', 'vote_count',
       'popularity', 'poster_path', 'backdrop_path', 'overview'],
      dtype='object')

In [5]:
mart.head(1)

,asset_id,movie_id,title,original_title,genre,adult,runtime,release_year,release_date,actors,director,orgnl_cntry,original_language,vote_average,vote_count,popularity,poster_path,backdrop_path,overview
0,cjc|M4820686LSGE28658201,11,스타워즈 4-새로운 희망,Star Wars,"모험, 액션, SF",False,121.0,1977.0,1977-05-25,"마크 해밀,캐리 피셔,해리슨 포드,알렉 기네스,피터 커싱",조지 루카스,미국,en,8.204,20724.0,118.411,/7XFfURIFCJxN1mfBg0SAjk5yGzg.jpg,/4qCqAdHcNKeAHcK8tJ8wNJZa9cx.jpg,"공화국이 붕괴하고 제국이 수립된 뒤 20년, 제다이 기사단은 전멸하고 강력한 제국군..."


<br><hr>

### 02. orgnl_cntry 전처리
- '중화인민공화국', '중화민국' >> '중국'
- '타이' >> '태국'
- '소련' >> '러시a아'
- '서독' >> '독일'

In [14]:
country_mapping = {
    "중화인민공화국": "중국",
    "중화민국": "중국",
    "타이": "태국",
    "소련": "러시아",
    "서독": "독일"
}

In [15]:
# 'orgnl_cntry' 열의 값을 매핑에 따라 변경
mart['orgnl_cntry'] = mart['orgnl_cntry'].replace(country_mapping)

In [20]:
# 공백 제거 및 나라 순서 통일
def clean_country_values(value):
    # 값이 문자열인지 확인 (null값 또는 숫자는 건너뜀)
    if isinstance(value, str):
        value = value.replace(" ", "")
        countries = value.split(",")
        sorted_countries = ",".join(sorted(countries))
        return sorted_countries
    return value

# orgnl_cntry 열에 적용
mart['orgnl_cntry'] = mart['orgnl_cntry'].apply(clean_country_values)


In [25]:
mart['orgnl_cntry'].value_counts()

orgnl_cntry
미국                  4789
대한민국                3411
일본                  1236
중국                   907
홍콩                   636
                    ... 
독일,미국,스페인,영국,프랑스       1
독일,벨기에,캐나다,프랑스         1
미국,인도                  1
덴마크,미국,프랑스             1
키프로스                   1
Name: count, Length: 205, dtype: int64

<br><hr>

### 03. actors 전처리

- '-' >> null값으로 변경
- ',' (반점) 뒤 공백 제거

In [30]:
print("actor 열의 null값:", mart['actors'].isnull().sum())
mart['actors'].value_counts()

actor 열의 null값: 0


actors
-                                                              64
금초군,하가경,범홍헌                                                     5
신밧드,사나,알리,라자크                                                   4
한삭,유관성                                                          4
알렉산드라 보르티취,일리야 말라닌,데니스 쉬베도프                                     4
                                                               ..
송혜교,남지현,송상우,송창의,조미혜,박은진,김현승,박준열,김한얼,박태훈,윤방수,이종찬,김주엽,김지영,김수안     1
공형진,김주혁,이윤지,이시영,오정세,조성걸,최정화,김형근,조달환,배슬기,조한철,김건우,정광재,장정숙,임동현     1
스티브 오스틴,대니 트레조                                                  1
로즈 번,패트릭 윌슨,린 샤예,타이 심킨스,바바라 허쉬,다니엘 비서티                          1
러슬란 아스타미로브,말릭 압드라마노브                                            1
Name: count, Length: 14076, dtype: int64

In [43]:
# null값 변경
mart['actors'] = mart['actors'].replace(["-", "미상"], np.nan)
print("actor 열의 null값:", mart['actors'].isnull().sum())

actor 열의 null값: 68


In [44]:
# 'actors' 열에서 , 뒤의 공백 제거
mart['actors'] = mart['actors'].str.replace(r",\s+", ",", regex=True)

In [45]:
mart['actors'].value_counts()

actors
금초군,하가경,범홍헌                                                    5
신밧드,사나,알리,라자크                                                  4
한삭,유관성                                                         4
강수진,김승준,이정구,정미숙                                                4
이와시타 시마,카타세 리노                                                 4
                                                              ..
공형진,김주혁,이윤지,이시영,오정세,조성걸,최정화,김형근,조달환,배슬기,조한철,김건우,정광재,장정숙,임동현    1
스티브 오스틴,대니 트레조                                                 1
로즈 번,패트릭 윌슨,린 샤예,타이 심킨스,바바라 허쉬,다니엘 비서티                         1
황정민,엄정화,정성화,오나라,박정민,박사랑,임호걸,정원영,안은정,라미란,이대연,이한위,최우리,천보근,권병길    1
러슬란 아스타미로브,말릭 압드라마노브                                           1
Name: count, Length: 14072, dtype: int64

In [76]:
# mart.to_csv('mart_전처리중.csv', encoding='utf8', index=False)

<br><hr>

### 04. assetid의 개수와 title-movie_id 개수 비교

In [48]:
print(mart.shape[0])
print(mart['asset_id'].nunique())
print(mart[['title', 'movie_id']].drop_duplicates().shape[0])

14292
14292
14227


*65개(92-27)의 데이터 확인*

In [52]:
duplicates = mart[mart.duplicated(subset=['title', 'movie_id'], keep=False)]
duplicates.shape

(130, 19)

In [82]:
# 삭제할 값이 들어있는 데이터프레임 불러오기
# todelete = pd.read_csv('rows_to_delete.csv', encoding='utf8')

In [83]:
# mart_filtered = mart[~mart.isin(todelete.to_dict(orient="list")).all(axis=1)]
# difference_df = mart[~mart[['asset_id']].isin(todelete[['asset_id']].to_dict(orient="list")).all(axis=1)]
# difference_df = mart.loc[~mart.index.isin(todelete.index)]


print(difference_df.shape[0])
print(difference_df['asset_id'].nunique())
print(difference_df[['title', 'movie_id']].drop_duplicates().shape[0])

14224
14224
14224


<br><hr>

### 05. original_title 값 변경

- cjc|M4983848LSVJ32283301(asset_id) 의 original_title 변경 ชัตเตอร์ กดติดวิญญาณ

In [85]:
mart.loc[mart['asset_id'] == 'cjc|M4983848LSVJ32283301', 'original_title'] = "ชัตเตอร์ กดติดวิญญาณ"
mart[mart['asset_id']=='cjc|M4983848LSVJ32283301']

,asset_id,movie_id,title,original_title,genre,adult,runtime,release_year,release_date,actors,director,orgnl_cntry,original_language,vote_average,vote_count,popularity,poster_path,backdrop_path,overview
14256,cjc|M4983848LSVJ32283301,17111,셔터,ชัตเตอร์ กดติดวิญญาณ,"공포, 미스터리, 스릴러",False,97.0,2004.0,2004-09-09,"아난다 에버링햄,나타위라눗 통미,아치타 시카마나,운노프 찬파이불",반종 피산다나쿤,태국,th,7.293,1226.0,30.065,/igBvzh5ro0ySISCMUesLQsAweeY.jpg,/kyVSY4kJcC7ThHE6hQFXxtyaLJv.jpg,25살의 사진작가 '턴'과 그의 여자친구 '제인'. 대학동창의 결혼식에 다녀오던 길...


In [87]:
mart.to_csv("../data/movies_mart_0122.csv", encoding='utf8', index=False)